# Imports

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, make_scorer
from sklearn.model_selection import StratifiedKFold

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.utils.class_weight import compute_class_weight

from sklearn.pipeline import make_pipeline


from sklearn.ensemble import ExtraTreesClassifier
 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

from category_encoders import TargetEncoder 
    
    
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder, MinMaxScaler
import sklearn
from scipy.stats import linregress
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
 
    
from sklearn import set_config 
set_config(display='diagram') 

In [2]:
# from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

# New-Hack. Задание

In [39]:
data = pd.read_csv('train.csv', low_memory=True )
submit = pd.read_csv('test.csv', low_memory=True )

2021-09-25 21:14:26,224 [14536] WARNING  py.warnings:109: [JupyterRequire] C:\Users\sasha\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



In [88]:
# data.price_type.hist()

# Look at data

In [43]:
data.head()

,city,floor,id,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,...,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,street,date,realty_type,price_type
0,Пермь,NaN,COL_0,57.998207,56.292797,4,19,35,52,0,...,5.762963,5.530612,1964.118519,1960.959184,Пермский край,32.0,S27289,2020-01-05,10,0
1,Шатура,NaN,COL_1,55.574284,39.543835,3,24,37,59,0,...,2.894366,3.527778,1952.321678,1957.222222,Московская область,280.0,S17052,2020-01-05,10,0
2,Ярославль,NaN,COL_2,57.619140,39.850525,1,30,67,128,0,...,6.141414,7.222222,1968.150000,1973.370370,Ярославская область,297.4,S16913,2020-01-05,110,0
3,Новокузнецк,NaN,COL_3,53.897083,87.108604,0,0,5,21,0,...,8.581081,9.000000,1992.716216,2014.000000,Кемеровская область,190.0,S10148,2020-01-05,110,0
4,Москва,NaN,COL_4,55.802590,37.487110,1,23,64,153,0,...,7.263889,5.684211,1963.229167,1960.500000,Москва,60.2,S1338,2020-01-05,10,0


# Clean Data

In [44]:
# KNNImputer(n_neighbors=2, weights ='distance')

In [45]:
def get_value(X, column, mode='mean', value=0, columns_for_reg=None):

    if mode == 'value':
        X.loc[X[X[column].isna()].index ,column] = value # заполнение значением
    elif mode == 'max':
        X.loc[X[X[column].isna()].index ,column] = X[column].max() # заполнение max
    elif mode == 'min':
        X.loc[X[X[column].isna()].index ,column] = X[column].min() # заполнение min
    elif mode == 'median':
        X.loc[X[X[column].isna()].index ,column] = X[column].median() # заполнение медианой
    elif mode == 'mode':
        X.loc[X[X[column].isna()].index, column] = X[column].mode()[0] # заполнение модой
    elif mode == 'indicator': # метод индикатора (принимает значение 1 при наличие пропуска и 0 в остальных случаях)
        X['ind_'+str(column)] = 0
        X.loc[X[X[column].isna()].index, 'ind_'+str(column)] = 1
        X.loc[X[X[column].isna()].index, column] = 0
    elif mode == 'linreg': # линейная регрессия
        if columns_for_reg is None:
            cols = list(X.select_dtypes([np.number]).columns)
            cols.remove(column)
        else:
            cols = columns_for_reg
        X_tmp = X.dropna()
        m = LinearRegression().fit(X_tmp[cols], X_tmp[column])
        X.loc[X[X[column].isna()].index, column] = m.predict(X[X[column].isna()][cols])
    else:
        X.loc[X[X[column].isna()].index, column] = X[column].mean() # по умолчанию среднее значение
    return X.head(3)

def get_one_hot(X, cols):
    for each in cols:
        dummies = pd.get_dummies(X[each], prefix=each, drop_first=False)
        X = pd.concat([X, dummies], axis=1)
    return X


def cleaning(DataFrame):
    get_value(DataFrame, 'floor', mode='value', value=1) 

    for i in range(len(DataFrame)): #Странная логика
        if isinstance(DataFrame['floor'][i], str):
            DataFrame['floor'][i] = 1

            
    # Есть инпутер на этот случай
#     columns = ['reform_house_population_1000', 'reform_house_population_500',
#                'reform_house_population_1000', 'reform_house_population_500', 
#                'reform_mean_floor_count_1000' ,      'reform_mean_floor_count_500' ,
#                'reform_mean_year_building_1000'  ,      'reform_mean_year_building_500']
    
#     for col in columns:
#         get_value(DataFrame, col, mode='mean')


    DataFrame = DataFrame.drop("city", axis=1)
    DataFrame = DataFrame.drop("id", axis=1)
    DataFrame = DataFrame.drop("street", axis=1)
    DataFrame = DataFrame.drop("date", axis=1)
    DataFrame = DataFrame.drop("osm_city_nearest_name", axis=1)


    d = dict(DataFrame['region'].value_counts())

    for i, reg in enumerate(d):
        d[reg] = i

    DataFrame['region'] = DataFrame['region'].map(d)
    
    #get_one_hot(X_train, ['region'])
    #DataFrame = DataFrame.drop("region", axis=1)

    DataFrame['osm_city_nearest_population'].fillna(DataFrame['osm_city_nearest_population'].mean(), inplace=True)
    DataFrame['floor'] = DataFrame['floor'].astype('int')
    
    return DataFrame




In [59]:
# for i in data.columns:
#     if i not in submit.columns:
#         print(i)


train = cleaning(data)
test = cleaning(submit)

In [63]:
for i in train.columns:
    if i not in test.columns:
        print(i)

test.shape, train.shape

per_square_meter_price


((2974, 71), (279792, 72))

In [65]:
# train.info()

In [66]:
# data['region'].value_counts()

# Генерация новых признаков  (пока не готово)

In [67]:

# def generate_feature(data):
#     l1 = len(data.columns)
    
#     def get_lin_k_l(x):
#         filtered = x[x > 0.4].reset_index(drop=True)
#         try: m = filtered.argmax()
#         except: return -1
        
#         LH = filtered[:m]
#         slope1, intercept1, _, _, _ = linregress(np.arange(len(LH)), LH)
#         return slope1*100 

#     def get_lin_k_r(x):
#         filtered = x[x > 0.4].reset_index(drop=True)
#         try: m = filtered.argmax()
#         except: return -1

#         RH = filtered[m:]
#         slope2, intercept2, _, _, _ = linregress(np.arange(len(RH)), RH)
#         return slope2*100 
     
#     def get_data(s):
#         return int(s.split()[-1])
    
#     def get_LH_argmax(x):
#         filtered = x.reset_index(drop=True)
#         LH = filtered[:250]
#         LH_argmax = LH.argmax()    
#         return LH_argmax 
#     def get_RH_argmax(x):
#         filtered = x.reset_index(drop=True)
#         RH = filtered[250:]
#         RH_argmax = RH.argmax()+250        
#         return RH_argmax

#     def get_delta_argmax(x):
#         filtered = x.reset_index(drop=True)
#         LH = filtered[:250]
#         RH = filtered[250:]
#         LH_argmax, RH_argmax = LH.argmax(), RH.argmax()+250         
#         delta_argmax = RH_argmax-LH_argmax

#         return delta_argmax 
#     def get_LH_max(x):
#         filtered = x.reset_index(drop=True)
#         LH,RH = filtered[:250],filtered[250:]
#         LH_argmax, RH_argmax = LH.argmax(), RH.argmax()+250            
#         LH_max = filtered[LH_argmax] 
#         return LH_max
#     def get_RH_max(x):
#         filtered = x.reset_index(drop=True)
#         LH,RH = filtered[:250],filtered[250:]
#         LH_argmax, RH_argmax = LH.argmax(), RH.argmax()+250           
#         RH_max = filtered[RH_argmax]
#         return RH_max


#     def get_less_left(x):
#         filtered = x.reset_index(drop=True)
#         argma = filtered.argmax()
#         Left_H = filtered[:argma]
#         Less_left_N = len(Left_H[Left_H < 0.1])
#         return Less_left_N
    
    
#     def get_less_left_sum(x):
#         filtered = x.reset_index(drop=True)
#         argma = filtered.argmax()
#         Left_H = filtered[:argma]
#         Less_left_sum = np.sum(Left_H[Left_H < 0.1])
#         return Less_left_sum
   
 
#     data['max_day'] =data['max_day'].apply(get_data)# день через который начинается пик числом
 
 
    
#     l2 = len(data.columns)
#     print('Сгенериривал еще: ', l2-l1, 'фичей')
    
#     generated_features = ['LH_argmax','RH_argmax','delta_argmax','LH_max','RH_max',
#                           'width_75', 'width_60', 'width_70','width_80', 'width_90',
#                           'less20', 'less10', 'max_day','max', 'plato_2_4',
#                           'less_left_10','less_left_sum']
    
#     return data, generated_features

 
# train_extr_gener, generated_features = generate_feature(train_extr)
# submit_extr_gener, generated_features = generate_feature(submit_extr)


# Отбор признаков (пока не готово)

In [68]:
# import pandas as pd
# from tsfresh import extract_features, select_features

# def sel_features(features, fdr_level = 0.01):
#     train_x = features.drop( ['y'], axis = 1 )
#     train_y = features.y
#     print("selecting features...")
#     train_features_selected = select_features(train_x, train_y, fdr_level = fdr_level, n_jobs=8)
#     print("selected {} features.".format( len(train_features_selected.columns )))
#     train = train_features_selected.copy(deep=True)
#     features_selected_columns = train.columns
#     train['y'] = train_y
#     return train, features_selected_columns


# train_extr_sel, train_features_selected = sel_features(train_extr_gener)    
# submit_extr_sel  = submit_extr_gener[train_features_selected]
 

# Визуализация данных

# Подготовка данных для Pipelin'ов

In [16]:
# x_train = train_extr_sel.drop( 'y', axis = 1 )
# y_train = train_extr_sel.y  

# Piplines

In [17]:
target = 'per_square_meter_price' 
    
    
numerical_features = ['floor', 'lat', 'lng', 'osm_amenity_points_in_0.001',
       'osm_amenity_points_in_0.005', 'osm_amenity_points_in_0.0075',
       'osm_amenity_points_in_0.01', 'osm_building_points_in_0.001',
       'osm_building_points_in_0.005', 'osm_building_points_in_0.0075',
       'osm_building_points_in_0.01', 'osm_catering_points_in_0.001',
       'osm_catering_points_in_0.005', 'osm_catering_points_in_0.0075',
       'osm_catering_points_in_0.01', 'osm_city_closest_dist',
       'osm_city_nearest_population', 'osm_crossing_closest_dist',
       'osm_crossing_points_in_0.001', 'osm_crossing_points_in_0.005',
       'osm_crossing_points_in_0.0075', 'osm_crossing_points_in_0.01',
       'osm_culture_points_in_0.001', 'osm_culture_points_in_0.005',
       'osm_culture_points_in_0.0075', 'osm_culture_points_in_0.01',
       'osm_finance_points_in_0.001', 'osm_finance_points_in_0.005',
       'osm_finance_points_in_0.0075', 'osm_finance_points_in_0.01',
       'osm_healthcare_points_in_0.005', 'osm_healthcare_points_in_0.0075',
       'osm_healthcare_points_in_0.01', 'osm_historic_points_in_0.005',
       'osm_historic_points_in_0.0075', 'osm_historic_points_in_0.01',
       'osm_hotels_points_in_0.005', 'osm_hotels_points_in_0.0075',
       'osm_hotels_points_in_0.01', 'osm_leisure_points_in_0.005',
       'osm_leisure_points_in_0.0075', 'osm_leisure_points_in_0.01',
       'osm_offices_points_in_0.001', 'osm_offices_points_in_0.005',
       'osm_offices_points_in_0.0075', 'osm_offices_points_in_0.01',
       'osm_shops_points_in_0.001', 'osm_shops_points_in_0.005',
       'osm_shops_points_in_0.0075', 'osm_shops_points_in_0.01',
       'osm_subway_closest_dist', 'osm_train_stop_closest_dist',
       'osm_train_stop_points_in_0.005', 'osm_train_stop_points_in_0.0075',
       'osm_train_stop_points_in_0.01', 'osm_transport_stop_closest_dist',
       'osm_transport_stop_points_in_0.005',
       'osm_transport_stop_points_in_0.0075',
       'osm_transport_stop_points_in_0.01',
       'reform_count_of_houses_1000', 'reform_count_of_houses_500',
       'reform_house_population_1000', 'reform_house_population_500',
       'reform_mean_floor_count_1000', 'reform_mean_floor_count_500',
       'reform_mean_year_building_1000', 'reform_mean_year_building_500',
       'total_square', 'realty_type', 'price_type']

categorical_features =  ['region']
# hm_numerical_features = [] #hand_made_num

In [18]:
numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer() ),
                                        ('scaler',  StandardScaler()),
                                       # ('fs', SelectKBest(score_func=f_classif, k=190))
                                       ])

# Применение SimpleImputer, а затем OneHotEncoder
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer()),
                                     #     ('onehot', OneHotEncoder(handle_unknown='ignore'),
                                          ('TarEnc', TargetEncoder(handle_unknown='ignore'))])


# Собираем воедино трансформеры для числовых и категориальных признаков
data_transformer = ColumnTransformer(transformers=[
                                    ('numerical', numerical_transformer, numerical_features),
                                    ('categorical', categorical_transformer, categorical_features),
                                    ])

# Создание конвейера препроцессора, который сначала преобразует данные и затем применяет PCA.
preprocessor = Pipeline(steps=[('data_transformer', data_transformer)])#, 
preprocessor                   

Pipeline(steps=[('data_transformer',
                 ColumnTransformer(transformers=[('numerical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['floor', 'lat', 'lng',
                                                   'osm_amenity_points_in_0.001',
                                                   'osm_amenity_points_in_0.005',
                                                   'osm_amenity_points_in_0.0075',
                                                   'osm_amenity_points_in_0.01',
                                                   'osm_building_points_in_0.001',
                                                   'osm_building_points_in...
                                                   'osm_culture_points_in_0.001',
                                                   'osm_culture_points_in_0.005',
                                                   'osm_culture_points_in_0.0075',
                                                   'osm_culture_points_in_0.01',
                                                   'osm_finance_points_in_0.001',
                                                   'osm_finance_points_in_0.005',
                                                   'osm_finance_points_in_0.0075',
                                                   'osm_finance_points_in_0.01', ...]),
                                                 ('categorical',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('TarEnc',
                                                                   TargetEncoder(handle_unknown='ignore'))]),
                                                  ['region'])]))])

# Model in piplines

In [19]:
from sklearn.ensemble import RandomForestRegressor

# Create a based model
rf = RandomForestRegressor()


model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', rf)])

model

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['floor',
                                                                    'lat',
                                                                    'lng',
                                                                    'osm_amenity_points_in_0.001',
                                                                    'osm_amenity_points_in_0.005',
                                                                    'osm_amenity_points_in_0.0075',
                                                                    'osm_amenity_points_in_0.01',
                                                                    'osm_building_points_i...
                                                                    'osm_culture_points_in_0.0075',
                                                                    'osm_culture_points_in_0.01',
                                                                    'osm_finance_points_in_0.001',
                                                                    'osm_finance_points_in_0.005',
                                                                    'osm_finance_points_in_0.0075',
                                                                    'osm_finance_points_in_0.01', ...]),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('TarEnc',
                                                                                    TargetEncoder(handle_unknown='ignore'))]),
                                                                   ['region'])]))])),
                ('classifier', RandomForestRegressor())])

In [36]:
# model.fit(x_train, y_train)

In [24]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    #'classifier__bootstrap': [True],
    'classifier__max_depth': [7],
    'classifier__max_features': [5],
#     'classifier__min_samples_leaf': [3, 4, 5],
#     'classifier__min_samples_split': [8, 10, 12],
    'classifier__n_estimators': [300]
}


# Instantiate the grid search model
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = 5, verbose = 2) #Проверь колличество ядер
param_grid

{'classifier__max_depth': [5, 7],
 'classifier__max_features': [4, 5],
 'classifier__n_estimators': [300, 500]}

In [25]:
# from sklearn.model_selection import RandomizedSearchCV
 
# clf = RandomizedSearchCV(logistic, distributions, random_state=0)

In [70]:
# Fit the grid search to the data
grid_search.fit(train.drop(target, axis=1),
                train[target])

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  24 out of  24 | elapsed:  9.9min finished
2021-09-25 21:32:03,823 [14536] WARNING  py.warnings:109: [JupyterRequire] C:\Users\sasha\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):



GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessor',
                                        Pipeline(steps=[('data_transformer',
                                                         ColumnTransformer(transformers=[('numerical',
                                                                                          Pipeline(steps=[('imputer',
                                                                                                           SimpleImputer()),
                                                                                                          ('scaler',
                                                                                                           StandardScaler())]),
                                                                                          ['floor',
                                                                                           'lat',
                                                                                           'lng',
                                                                                           'osm_amenity_points_in_0.001',
                                                                                           'osm_amenity_points_in_0.005',
                                                                                           'osm_amenity_points_in_0.0075',
                                                                                           'osm_amenity_points_in_...
                                                                                           'osm_finance_points_in_0.0075',
                                                                                           'osm_finance_points_in_0.01', ...]),
                                                                                         ('categorical',
                                                                                          Pipeline(steps=[('imputer',
                                                                                                           SimpleImputer()),
                                                                                                          ('TarEnc',
                                                                                                           TargetEncoder(handle_unknown='ignore'))]),
                                                                                          ['region'])]))])),
                                       ('classifier',
                                        RandomForestRegressor())]),
             n_jobs=5,
             param_grid={'classifier__max_depth': [5, 7],
                         'classifier__max_features': [4, 5],
                         'classifier__n_estimators': [300, 500]},
             verbose=2)

In [75]:
print(grid_search.best_params_)
#print(grid_search.best_score_)

best_grid = grid_search.best_estimator_
# best_grid

# grid_accuracy = evaluate(best_grid, test_features, test_labels)
# print('Improvement of {:0.2f}%.'.format( 100 * (grid_accuracy - base_accuracy) / base_accuracy))

{'classifier__max_depth': 7, 'classifier__max_features': 5, 'classifier__n_estimators': 500}


In [76]:
# Create a based model
rf = RandomForestRegressor(max_depth = 7, max_features = 5, n_estimators=300, n_jobs = -1)


model = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', rf)])

model

Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['floor',
                                                                    'lat',
                                                                    'lng',
                                                                    'osm_amenity_points_in_0.001',
                                                                    'osm_amenity_points_in_0.005',
                                                                    'osm_amenity_points_in_0.0075',
                                                                    'osm_amenity_points_in_0.01',
                                                                    'osm_building_points_i...
                                                                    'osm_culture_points_in_0.01',
                                                                    'osm_finance_points_in_0.001',
                                                                    'osm_finance_points_in_0.005',
                                                                    'osm_finance_points_in_0.0075',
                                                                    'osm_finance_points_in_0.01', ...]),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('TarEnc',
                                                                                    TargetEncoder(handle_unknown='ignore'))]),
                                                                   ['region'])]))])),
                ('classifier',
                 RandomForestRegressor(max_depth=7, max_features=5,
                                       n_estimators=300, n_jobs=-1))])

In [77]:
model.fit(train.drop(target, axis=1), train[target])

2021-09-25 21:35:13,624 [14536] WARNING  py.warnings:109: [JupyterRequire] C:\Users\sasha\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):



Pipeline(steps=[('preprocessor',
                 Pipeline(steps=[('data_transformer',
                                  ColumnTransformer(transformers=[('numerical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('scaler',
                                                                                    StandardScaler())]),
                                                                   ['floor',
                                                                    'lat',
                                                                    'lng',
                                                                    'osm_amenity_points_in_0.001',
                                                                    'osm_amenity_points_in_0.005',
                                                                    'osm_amenity_points_in_0.0075',
                                                                    'osm_amenity_points_in_0.01',
                                                                    'osm_building_points_i...
                                                                    'osm_culture_points_in_0.01',
                                                                    'osm_finance_points_in_0.001',
                                                                    'osm_finance_points_in_0.005',
                                                                    'osm_finance_points_in_0.0075',
                                                                    'osm_finance_points_in_0.01', ...]),
                                                                  ('categorical',
                                                                   Pipeline(steps=[('imputer',
                                                                                    SimpleImputer()),
                                                                                   ('TarEnc',
                                                                                    TargetEncoder(handle_unknown='ignore'))]),
                                                                   ['region'])]))])),
                ('classifier',
                 RandomForestRegressor(max_depth=7, max_features=5,
                                       n_estimators=300, n_jobs=-1))])

In [78]:
preds = model.predict(test)

2021-09-25 21:35:46,831 [14536] WARNING  py.warnings:109: [JupyterRequire] C:\Users\sasha\anaconda3\lib\site-packages\sklearn\compose\_column_transformer.py:437: FutureWarning: Given feature/column names or counts do not match the ones for the data given during fit. This will fail from v0.24.
  warnings.warn("Given feature/column names or counts do not match "



array([63332.08311565, 63605.73545168, 70347.98884838, ...,
       65244.68301694, 77913.96422563, 76148.92389303])

In [84]:
sub  = pd.read_csv('test_submission.csv')
sub[target] = preds

,id,per_square_meter_price
0,COL_289284,63332.083116
1,COL_289305,63605.735452
2,COL_289318,70347.988848
3,COL_289354,77895.422505
4,COL_289399,93913.399699
...,...,...
2969,COL_455089,75706.014608
2970,COL_455212,73798.355049
2971,COL_455261,65244.683017
2972,COL_455381,77913.964226


In [83]:
sub.to_csv('best.csv') # 0.7456 the best sub!!!!!!

# Анализ ошибок

,floor,lat,lng,osm_amenity_points_in_0.001,osm_amenity_points_in_0.005,osm_amenity_points_in_0.0075,osm_amenity_points_in_0.01,osm_building_points_in_0.001,osm_building_points_in_0.005,osm_building_points_in_0.0075,...,reform_house_population_1000,reform_house_population_500,reform_mean_floor_count_1000,reform_mean_floor_count_500,reform_mean_year_building_1000,reform_mean_year_building_500,region,total_square,realty_type,price_type
0,1,57.998207,56.292797,4,19,35,52,0,0,0,...,2503.0,765.0,5.762963,5.530612,1964.118519,1960.959184,14,32.000000,10,0
1,1,55.574284,39.543835,3,24,37,59,0,0,0,...,1336.0,514.0,2.894366,3.527778,1952.321678,1957.222222,3,280.000000,10,0
2,1,57.619140,39.850525,1,30,67,128,0,0,1,...,1883.0,573.0,6.141414,7.222222,1968.150000,1973.370370,29,297.400000,110,0
3,1,53.897083,87.108604,0,0,5,21,0,0,0,...,1801.0,54.0,8.581081,9.000000,1992.716216,2014.000000,18,190.000000,110,0
4,1,55.802590,37.487110,1,23,64,153,0,1,1,...,3090.0,619.0,7.263889,5.684211,1963.229167,1960.500000,0,60.200000,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
279787,1,56.459183,84.979334,2,33,111,222,0,0,1,...,3008.0,979.0,5.714286,5.882353,1972.260870,1973.460000,39,358.320073,10,1
279788,1,59.936954,30.356383,10,274,718,1340,0,2,5,...,7661.0,1659.0,4.719388,4.706667,1876.994898,1873.186667,2,119.637556,110,1
279789,1,54.729233,20.514968,0,12,34,84,0,2,2,...,1225.0,460.0,3.950413,4.885714,1964.258333,1970.571429,21,312.789725,10,1
279790,1,55.360680,86.081460,5,57,100,134,0,0,0,...,1649.0,429.0,4.691489,4.125000,1957.425532,1954.625000,18,89.201305,110,1


# Модель ExraTree + FeatureExctractor как бинарный классификатор для похожих классов

In [38]:
ETC = ExtraTreesClassifier(max_depth=16, 
                           n_estimators=385,  
                            n_jobs=-1,
                           class_weight='balanced',
                           random_state=42)
 
categorical_features =  cat_cols
numerical_features = num_cols
 

hand_made_numerical_features = ['max_day', 'max', 'RH_argmax', 
                         'width_70', 'width_60', 'width_80',
                         'less10', 'less_left_10','less_left_sum']   

 
numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer() ),
                                        ('scaler',  StandardScaler()),
                                        ('fs', SelectKBest(score_func=f_classif, k=195))])
                                       

hand_made_numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer()),
                                           ('scaler',  StandardScaler())])


# Применение SimpleImputer, а затем OneHotEncoder
categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer()),
                                          ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Собираем воедино трансформеры для числовых и категориальных признаков
data_transformer = ColumnTransformer(transformers=[
                                    ('numerical', numerical_transformer, numerical_features),
                                    ('categorical', categorical_transformer, categorical_features),
                                    ('hand_made', hand_made_numerical_transformer, hand_made_numerical_features)])
                                    

# Создание конвейера препроцессора, который сначала преобразует данные 
preprocessor = Pipeline(steps=[('data_transformer', data_transformer)])  
                              

classifier_ETC_binar  = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', ETC)])

 
classifier_ETC_binar 

# Catboost

In [34]:
# Применяем SimpleImputer и будем искать различные скейлеры с помощью GridSearchCV
numerical_transformer = Pipeline(steps=[('imputer', SimpleImputer() ),
#                                         ('scaler',  StandardScaler()),
                                        ('fs', SelectKBest(score_func=f_classif, k=190)) ])                                     

categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer()),
#                                           ('onehot', OneHotEncoder(handle_unknown='ignore'))
                                         ])

series_transformer = Pipeline(steps=[('imputer_ser', KNNImputer(n_neighbors=9, weights="uniform")),
#                                     ('scaler',  StandardScaler()),
#                                     ('fs', SelectKBest(score_func=f_classif, k=190)),
                                    ])                                      

# Собираем воедино трансформеры для числовых и категориальных признаков
data_transformer = ColumnTransformer(transformers=[
                                    ('numerical', numerical_transformer, numerical_features),
                                    ('categorical', categorical_transformer, categorical_features),#])
                                     ('hand_made', hm_numerical_transformer, hm_numerical_features),#])
                                    ('series', series_transformer, series_features),])
                                     

# Создание конвейера препроцессора, который сначала преобразует данные и затем применяет PCA.
preprocessor = Pipeline(steps=[('data_transformer', data_transformer)])
                               #('reduce_dim', TruncatedSVD())])
classifier_ctb = Pipeline(steps=[('preprocessor', preprocessor),
                              ('classifier', catboost.CatBoostClassifier(iterations=1000, random_state=42))])
#max_depth=6, n_estimators=305
#max_depth = 5, n_estimators=350, colsample_bytree = 0.7
classifier_ctb

In [24]:
X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, 
                 stratify=y_train, random_state=42)

In [25]:
classifier_ctb.fit(x_train, y_train)  
classifier_ctb_preds = classifier_ctb.predict(submit_extr_sel)  

Learning rate set to 0.5
0:	learn: 1.5564657	total: 2.58s	remaining: 0us


In [26]:
classifier_ctb_preds_resh =  classifier_ctb_preds[:, 0]

# Блэндинг моделей Extra Tree (без FE) и Catboost

In [27]:
a = 0.57 
st_sol = pd.Series(((classifier_ctb.predict_proba(submit_extr_sel) * a
                     + classifier_ETC.predict_proba(submit_extr_sel) * (1-a)).argmax(1) + 1))

# Теперь исправим ошибки стекинга с помощь Extra Tree + Feature Extractor

# Подготовка сабмита

In [31]:
pd.Series(new_aswer).to_csv('best.csv', index=False, header=['Culture']) # 0.7456 the best sub!!!!!!

# Замечание

При решении этого хакатона, конечно, было проверенно намного больше гипотез, чем в этом ноутбуке, но для простоты восприятия, была сделана выжимка кода и оставлены только те фрагменты, которые сработали и дали лучший результат на борде. 

Были убраны модели, которые показали плохие результаты (RandomForest, KNN, XGBoost и тп). 
Так же удалена оптимизация по параметрами моделей с помощью GridSerchCV.